In [ ]:
import os
from diffusers import StableDiffusionInpaintPipeline
from datetime import datetime
import torch
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Set up the model path - using the pre-installed Fluently v3 inpainting model
model_path = '/jupyterlab/hf/hub/models--fluently--Fluently-v3-inpainting/snapshots/9983a35adbd27ba2a56b392e46b237535af176a8'

# Load the inpainting pipeline
print("Loading Fluently v3 inpainting pipeline...")
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16
)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)
print(f"Inpainting pipeline loaded on {device}")

In [ ]:
# Create a simple base image for inpainting demonstration
def create_demo_image():
    """Create a simple demo image with a basic scene"""
    # Create a 512x512 image with a simple landscape
    img = Image.new('RGB', (512, 512), color='lightblue')  # Sky
    draw = ImageDraw.Draw(img)

    # Draw ground
    draw.rectangle([0, 350, 512, 512], fill='lightgreen')

    # Draw a simple house
    draw.rectangle([200, 250, 350, 350], fill='brown')  # House base
    draw.polygon([180, 250, 275, 180, 370, 250], fill='red')  # Roof

    # Draw a window (this will be our inpainting target)
    draw.rectangle([240, 280, 280, 320], fill='white')

    return img

# Create mask for inpainting (white areas will be inpainted)
def create_mask():
    """Create a mask for the window area"""
    mask = Image.new('L', (512, 512), 0)  # Black background
    draw = ImageDraw.Draw(mask)
    # White circle in the center-left area (this will be inpainted)
    draw.ellipse([100, 200, 200, 300], fill=255)
    return mask

# Create demo images
base_image = create_demo_image()
mask_image = create_mask()

# Display the base image and mask
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(base_image)
axes[0].set_title("Base Image")
axes[0].axis('off')

axes[1].imshow(mask_image, cmap='gray')
axes[1].set_title("Inpainting Mask (white area will be replaced)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Perform inpainting with different prompts
inpainting_prompts = [
    "a beautiful tree with green leaves",
    "a colorful flower garden",
    "a small pond with ducks swimming"
]

inpainted_images = []

for i, prompt in enumerate(inpainting_prompts):
    print(f"Generating inpainting {i+1}/3: {prompt}")

    # Perform inpainting
    result = pipe(
        prompt=prompt,
        image=base_image,
        mask_image=mask_image,
        num_inference_steps=20,
        guidance_scale=7.5,
        strength=0.75
    ).images[0]

    inpainted_images.append(result)

    # Save the result
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"inpainting_{i+1}_{timestamp}.png"
    result.save(filename)
    print(f"Saved: {filename}")

print("All inpainting results generated successfully!")

In [ ]:
# Display all inpainting results
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Show original
axes[0, 0].imshow(base_image)
axes[0, 0].set_title("Original Image")
axes[0, 0].axis('off')

# Show inpainting results
positions = [(0, 1), (1, 0), (1, 1)]
for i, (image, prompt) in enumerate(zip(inpainted_images, inpainting_prompts)):
    row, col = positions[i]
    axes[row, col].imshow(image)
    axes[row, col].set_title(f"Inpainting: {prompt}")
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Test model capabilities with a more complex scene
print("Testing inpainting with a complex prompt...")

complex_prompt = "a magical portal with swirling purple and blue energy, fantasy style"

complex_result = pipe(
    prompt=complex_prompt,
    image=base_image,
    mask_image=mask_image,
    num_inference_steps=25,
    guidance_scale=8.0,
    strength=0.8
).images[0]

# Display the complex result
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(base_image)
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(mask_image, cmap='gray')
axes[1].set_title("Mask")
axes[1].axis('off')

axes[2].imshow(complex_result)
axes[2].set_title("Inpainted: Magical Portal")
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Save the complex result
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
complex_filename = f"complex_inpainting_{timestamp}.png"
complex_result.save(complex_filename)
print(f"Complex inpainting saved: {complex_filename}")